In [1]:
import pygame as pg
from PIL import Image
from collections import defaultdict

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [17]:
def tictactoe():
    def create_button_rects(button_rects):
        for i in range(9):
            rect = pg.Rect(0,0,button_size,button_size)
            rect.center = (int((i%3)*button_size+button_size/2),int((i//3)*button_size+button_size/2))
            button_rects.append(rect.copy())
    def draw_canvas():
        pg.draw.rect(screen,button_bg,pg.Rect(0,0,width,3*button_size))
        pg.draw.line(screen,black,(button_size,0),(button_size,3*button_size),line_width)
        pg.draw.line(screen,black,(2*button_size,0),(2*button_size,3*button_size),line_width)
        pg.draw.line(screen,black,(0,button_size),(width,button_size),line_width)
        pg.draw.line(screen,black,(0,height/2),(width,height/2),line_width)
        pg.draw.rect(screen,black,pg.Rect(0,3*button_size,3*button_size,3))
    def initialize_image_list():
        nonlocal image_list
        for i in range(9):
            image_list[i]=None
    def get_button_index(mousepos):
        return mousepos[0]//button_size+(mousepos[1]//button_size)*3
    def get_image_rect(rect,border):
        temp = rect.copy()
        temp.width-=border
        temp.height-=border
        temp.center = rect.center
        return temp
    def button_click(index,turn):
        if turn ==0:
            image_list[index]=X
            screen.blit(X,get_image_rect(button_rects[index],10))
        elif turn == 1:
            image_list[index]=O
            screen.blit(O,get_image_rect(button_rects[index],10))
    def convert_to_surface(image):
        mode = image.mode
        size = image.size
        data = image.tobytes()
        return pg.image.fromstring(data,size,mode)
    def iswon(position):
        for i in win[position]:
            result = True
            for j in i:
                result = result and image_list[j-1]==image_list[position-1]
            if result:
                return i
        return None
    def reset_routine():
        nonlocal turn
        draw_canvas()
        initialize_image_list()
        turn = 0
    def in_rect(rect,position):
        return position[0]>rect.left and position[0]<rect.right and position[1]>rect.top and position[1]<rect.bottom
    def change_pointer(rect):
        if in_rect(rect,pg.mouse.get_pos()):
            pg.mouse.set_cursor(pg.SYSTEM_CURSOR_HAND)
        else:
            pg.mouse.set_cursor(pg.SYSTEM_CURSOR_ARROW)

    def strike(buttons,turn):
        color = None
        if turn == 0:
            color = (240,0,0)
        if turn == 1:
            color = (0,128,255)
        start = None
        end = None
        length = None
        if buttons[0]==1 and buttons[1]==5 or buttons[0]==3 and buttons[1]==5:
            if buttons[0]==1:
                start = button_rects[buttons[0]-1].topleft
                end = button_rects[buttons[-1]-1].bottomright
            else:
                start = button_rects[buttons[0]-1].topright
                end = button_rects[buttons[-1]-1].bottomleft
            length = width*2**(1/2)
        else:
            if buttons[0] in (1,2,3):
                if buttons[0]==1 and buttons[1]==2:
                    start = button_rects[buttons[0]-1]
                    start = (start.left,start.centery)
                    end = button_rects[buttons[-1]-1]
                    end = (end.right,end.centery)
                else:
                    start = button_rects[buttons[0]-1]
                    start = (start.centerx,start.top)
                    end = button_rects[buttons[-1]-1]
                    end = (end.centerx,end.bottom)
            else:
                start = button_rects[buttons[0]-1]
                start = (start.left,start.centery)
                end = button_rects[buttons[-1]-1]
                end = (end.right,end.centery)
            length = width
        framerate = 80
        inc = length/framerate
        line_width = 4
        widthinc = 0
        heightinc = 0
        count = 0
        line_rect = pg.Rect(0,0,10,length)
        clock = pg.time.Clock()
        running = True
        while running:
            clock.tick(framerate)
            mouse = pg.mouse.get_pos()
            if count <framerate:
                if start[0]!=end[0]:
                    if start[0]>end[0]:
                        widthinc-=inc
                    else:
                        widthinc+=inc
                if start[1]!=end[1]:
                    if start[1]>end[1]:
                        heightinc-=inc
                    else:
                        heightinc+=inc
                pg.draw.line(screen,color,start,[start[0]+widthinc,start[1]+heightinc],5)
                count+=1
            change_pointer(reload_rect)
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    return True
                if event.type == pg.MOUSEBUTTONDOWN:
                    if in_rect(reload_rect,pg.mouse.get_pos()):
                        reset_routine()
                        return
            pg.display.flip()

    win_list = [[1,2,3],[4,5,6],[7,8,9],[1,4,7],[2,5,8],[3,6,9],[1,5,9],[3,5,7]]
    win = defaultdict(lambda:[])
    for i in win_list:
        for j in i:
            win[j].append(i)

    width = 300
    height = 400
    screen = pg.display.set_mode((width,height))
    pg.display.set_caption('Tic Tac Toe')
    screen.fill((230,230,230))
    black = (0,0,0)
    button_bg = (240,240,240)
    button_hover_bg = (245,245,245)
    button_size = 100
    line_width = 1
    button_rects = []
    create_button_rects(button_rects)
    active_button = -1
    recent_active_button = -1
    X = Image.open('Tic Tac Toe images\X.png')
    X = X.resize((button_size-10,button_size-10),Image.ANTIALIAS)
    X = convert_to_surface(X)
    O = Image.open('Tic Tac Toe images\O.png')
    O = O.resize((button_size-10,button_size-10),Image.ANTIALIAS)
    O = convert_to_surface(O)
    reload = Image.open('Tic Tac Toe images\\reload.png')
    reload = reload.resize((button_size//2,button_size//2),Image.ANTIALIAS)
    reload = convert_to_surface(reload)
    reload_rect = reload.get_rect()
    reload_rect.center = (width/2,3*button_size+button_size/2)
    screen.blit(reload,reload_rect)
    image_list = list(range(9))
    initialize_image_list()
    draw_canvas()
    turn = 0
    running = True
    while running:
        mousepos = pg.mouse.get_pos()
        active_button = get_button_index(mousepos)
        if not recent_active_button == active_button and active_button<9:
            pg.draw.rect(screen,button_hover_bg,get_image_rect(button_rects[active_button],line_width))
            if image_list[active_button]!=None:
                screen.blit(image_list[active_button],get_image_rect(button_rects[active_button],10))
        if not recent_active_button == active_button and recent_active_button < 9 and recent_active_button>-1:
            pg.draw.rect(screen,button_bg,get_image_rect(button_rects[recent_active_button],line_width))
            if image_list[recent_active_button]!=None:
                screen.blit(image_list[recent_active_button],get_image_rect(button_rects[recent_active_button],10))
        if active_button<9:
            recent_active_button = active_button
        else :
            recent_active_button = -1
        for event in pg.event.get():
            if event.type == pg.QUIT:
                running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                if active_button<9 and image_list[active_button]==None:
                    button_click(active_button,turn)
                    buttons = iswon(active_button+1)
                    if buttons!=None:
                        if strike(buttons,turn):
                            running = False
                        turn = 0
                        continue
                    turn= turn^1
                elif in_rect(reload_rect,pg.mouse.get_pos()):
                    reset_routine()
        change_pointer(reload_rect)
        pg.display.update()
    pg.quit()
tictactoe()